## Import libraries

In [1]:
import src.config as config

2025-06-14 11:06:19.916 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Repositories\ds-lstm-ibov


## ETL — Extract, Transform, Load of Raw Dataset

This section is responsible for the Extraction phase of the ETL process, pulling historical financial data from multiple sources as defined in the configuration file [dataset.yaml](../configs/dataset.yaml).

- Data Storage: Each dataset is stored locally in the data/raw/ directory, organized and saved individually as .csv files for traceability and versioning.
- Data Cleaning:
  - Missing values are treated using a standard cleaning strategy.
  - Features with low variance (threshold < 0.01) are removed to reduce noise and improve modeling efficiency. In this execution, no low-variance columns were found.

In [2]:
from src import dataset

dataset.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    asset = '^BVSP',
    asset_focus = 'Close',
    years = 10
    # -----------------------------------------
    )

2025-06-14 11:06:23.146 | INFO     | src.dataset:main:31 - Starting raw data loading...
2025-06-14 11:06:23.146 | INFO     | src.dataset:main:37 - Requesting information between 2015-06-17 and 2025-06-14
2025-06-14 11:06:23.146 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BOVESPA (^BVSP) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-14 11:06:27.091 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading S&P500 (^GSPC) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-14 11:06:29.992 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BITCOIN (BTC-USD) from yfinance...


...

2025-06-14 11:06:39.250 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading ACUCAR (SB=F) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-14 11:06:42.097 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC (11) from the Central Bank of Brazil API...
2025-06-14 11:07:04.766 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading CDI (12) from the Central Bank of Brazil API...
2025-06-14 11:07:26.085 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Anual (1178) from the Central Bank of Brazil API...
2025-06-14 11:07:47.442 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Meta_Anual (432) from the Central Bank of Brazil API...
2025-06-14 11:07:49.862 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading IPCA_Mensal (433) from the Central Bank of Brazil API...
2025-06-14 11:07:52.075 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading IGP_M_Mensal (189) from the Central Bank of Brazil API...
2025-06-14 11:07:54.351 | INFO     | src.utils.dataset.

INFO:src.utils.dataset.clean_strategy:Executing CleanMissingValues...
INFO:src.utils.dataset.clean_strategy:Executing CleanLowVariance with threshold=0.01...
INFO:src.utils.dataset.clean_strategy:Columns removed due to low variance: Index([], dtype='object')


2025-06-14 11:08:21.067 | SUCCESS  | src.dataset:main:63 - Raw data successfully loaded...
2025-06-14 11:08:21.289 | SUCCESS  | src.dataset:main:83 - Clean data successfully loaded...
2025-06-14 11:08:21.289 | INFO     | src.dataset:main:87 - Total time taken: 118.14 seconds


## Feature Engineering — Time Series Preparation

This part of the pipeline is responsible for transforming the cleaned dataset into structured features suitable for training time series models. It includes the following key steps:
- Feature Generation: Constructs relevant features based on historical market data.
- Dataset Splitting: The dataset is split into training and testing sets using a consistent strategy to preserve temporal structure.
- Time Series Windowing: Converts the sequential data into overlapping windows, enabling the model to learn temporal dependencies.
- Saving Artifacts: Both training and testing sets are stored for reproducibility, along with the transformation pipelines applied during preprocessing.

In [3]:
from src import features

features.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    train_dir = config.PROCESSED_DATA_DIR,
    test_dir = config.PROCESSED_DATA_DIR,
    # -----------------------------------------
)

2025-06-14 11:08:30.868 | INFO     | src.features:main:31 - Generating features from dataset...
2025-06-14 11:08:30.918 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-14 11:08:30.934 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...
2025-06-14 11:08:31.001 | SUCCESS  | src.features:main:46 - Saving train features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-14 11:08:31.018 | SUCCESS  | src.features:main:50 - Saving test features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-14 11:08:31.018 | SUCCESS  | src.features:main:54 - Features generation complete.
2025-06-14 11:08:31.018 | INFO     | src.features:main:56 - Saving transformers...
2025-06-14 11:08:31.018 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-14 11:08:31.018 | INFO     | src.features:main:72 - Total t

## Modeling

The modeling process begins with loading the training dataset. Next, the base model is constructed, and both the compilation and training strategies are defined. With the pipeline structure in place, the model is trained over 100 epochs using an iterative approach to adjust the weights.

During training, key metrics are monitored, including accuracy, loss (error), validation accuracy and loss (on unseen data), as well as the learning rate.

In [4]:
from src.modeling import train

train.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    X_path = config.PROCESSED_DATA_DIR / "X_train.npy",
    y_path = config.PROCESSED_DATA_DIR / "y_train.npy"
    # -----------------------------------------
)

2025-06-14 11:08:31.057 | INFO     | src.modeling.train:main:29 - Loading training dataset...
2025-06-14 11:08:31.084 | INFO     | src.modeling.train:main:37 - Building model...
2025-06-14 11:08:31.084 | INFO     | src.modeling.train:main:43 - Selecting compile strategy...
2025-06-14 11:08:31.084 | INFO     | src.modeling.train:main:46 - Selecting training strategy...
2025-06-14 11:08:31.084 | INFO     | src.modeling.train:main:49 - Building model training pipeline template...
2025-06-14 11:08:31.084 | INFO     | src.modeling.train:main:56 - Training model...
Epoch 1/100
3281/3281 ━━━━━━━━━━━━━━━━━━━━ 50s 13ms/step - accuracy: 0.6417 - loss: 0.1580 - val_accuracy: 0.3096 - val_loss: 0.0655 - learning_rate: 0.0010
Epoch 2/100
3281/3281 ━━━━━━━━━━━━━━━━━━━━ 49s 15ms/step - accuracy: 0.6449 - loss: 0.0434 - val_accuracy: 0.3205 - val_loss: 0.0587 - learning_rate: 0.0010
Epoch 3/100
3281/3281 ━━━━━━━━━━━━━━━━━━━━ 53s 16ms/step - accuracy: 0.6849 - loss: 0.0389 - val_accuracy: 0.3726 - val_

## Predict

In the prediction stage, the trained model is loaded along with the input data. The model then performs inference, generating predictions based on the provided data.


In [7]:
from src.modeling import predict
predict.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    preprocessor_path = config.PROCESSED_DATA_DIR / "preprocessor.pkl",
    model_path = config.MODELS_DIR / "Sequential_epoch89_loss0.0186.keras",  # Select the best model  
    postprocessor_path = config.PROCESSED_DATA_DIR / "postprocessor.pkl",
    output_path = config.PROCESSED_DATA_DIR / "dataset_report.csv",
    # -----------------------------------------
)

2025-06-14 12:36:57.549 | INFO     | src.modeling.predict:main:27 - Performing inference for model...
2025-06-14 12:36:57.644 | INFO     | src.modeling.predict:main:31 - Input data shape: (8, 45)
2025-06-14 12:36:57.654 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...


c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
2025-06-14 12:36:59.870 | SUCCESS  | src.modeling.predict:main:66 - Inference complete.


## Plot

Open the [Power BI Report]((../reports/pbi/amp-fynance.pbip)) and refresh data.